In [2]:
!pip install opencv-python
!pip install numpy
!pip install imutils

In [4]:
!mkdir BallTracking
!cd BallTracking


In [2]:
import cv2
import numpy as np
import imutils
import time

In [3]:
color_ranges = {
    'red': ((0, 100, 100), (10, 255, 255)),
    'blue': ((100, 150, 0), (140, 255, 255)),
    'green': ((40, 70, 70), (80, 255, 255)),
    'yellow': ((20, 100, 100), (30, 255, 255))

    # Add more colors if needed
}

In [4]:
def get_quadrant(x, y, width, height):
    if x < width // 2 and y < height // 2:
        return 1
    elif x >= width // 2 and y < height // 2:
        return 2
    elif x < width // 2 and y >= height // 2:
        return 3
    else:
        return 4

In [5]:
# Initialize the video capture
video_path = "AI Assignment video.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()


In [6]:
# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Prepare output video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, fps, (width, height))

# Initialize tracking data
tracking_data = []

# Initialize previous positions
previous_positions = {color: [] for color in color_ranges}

# Function to track balls and record events
def track_balls(frame, frame_time):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    for color_name, (lower, upper) in color_ranges.items():
        mask = cv2.inRange(hsv, lower, upper)
        cnts, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for c in cnts:
            if cv2.contourArea(c) < 500:
                continue
            ((x, y), radius) = cv2.minEnclosingCircle(c)
            if radius > 10:
                center_x = int(x)
                center_y = int(y)
                quadrant = get_quadrant(center_x, center_y, width, height)
                event_type = "Entry" if (center_x, center_y) not in previous_positions[color_name] else "Exit"
                previous_positions[color_name].append((center_x, center_y))
                tracking_data.append((frame_time, quadrant, color_name, event_type))
                text = f"{event_type} - {frame_time:.2f}s"
                cv2.putText(frame, text, (center_x, center_y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                cv2.circle(frame, (center_x, center_y), int(radius), (0, 255, 255), 2)
                cv2.putText(frame, f"Quadrant {quadrant}", (center_x, center_y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

# Main loop to process the video
frame_skip = int(fps / 2)  # Process every other frame to reduce load
start_time = time.time()
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if frame_count % frame_skip != 0:
        continue

    frame_time = time.time() - start_time
    track_balls(frame, frame_time)
    out.write(frame)
    cv2.imshow('Frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

# Write tracking data to a text file
with open('tracking_data.txt', 'w') as file:
    for data in tracking_data:
        file.write(f"{data[0]:.2f}, {data[1]}, {data[2]}, {data[3]}\n")

print("Processing complete. Output video and tracking data saved.")

Processing complete. Output video and tracking data saved.
